In [1]:
import numpy as np
import pandas as pd
import random as rd
from random import randint

#Считываем файл, заполняем списки ценности, объёмов и размеров
myList=[]
with open('14.txt') as f:
    for line in f:
        myList.append([float(x) for x in line.split()])
lngth=len(myList)


maxWeight=myList[0][0]
maxSize=myList[0][1]

value=[]
size=[]
weight=[]

for i in range (1,lngth):
    #ценность
    value.append(myList[i][2])
    #размер
    size.append(myList[i][1])
    #вес
    weight.append(myList[i][0])
print(myList,'\n')


[[13000.0, 12.0], [851.0, 0.9, 260.0], [1470.0, 0.7, 197.0], [146.0, 0.4, 123.0], [1544.0, 1.1, 207.0], [876.0, 1.1, 275.0], [1045.0, 0.5, 205.0], [240.0, 1.0, 141.0], [1581.0, 0.8, 298.0], [1701.0, 0.5, 253.0], [449.0, 0.5, 302.0], [458.0, 0.9, 277.0], [1110.0, 0.5, 351.0], [522.0, 0.7, 199.0], [1658.0, 1.1, 137.0], [215.0, 0.4, 373.0], [1437.0, 1.2, 324.0], [1690.0, 1.1, 146.0], [1537.0, 0.9, 389.0], [990.0, 0.6, 142.0], [221.0, 1.1, 318.0], [455.0, 0.5, 325.0], [1455.0, 0.7, 317.0], [452.0, 1.2, 326.0], [548.0, 0.4, 265.0], [264.0, 0.8, 113.0], [611.0, 0.9, 242.0], [1675.0, 0.9, 320.0], [456.0, 1.0, 186.0], [945.0, 0.9, 134.0], [935.0, 0.5, 295.0]] 



In [2]:
numGenes = len(myList) - 1 #количество генов в наборе
population_size = 200 #величина популяции

def weight_checking(set_of_items): #проверка набора на соответствие весу
    sumWeight = 0
    for i in range(numGenes):
        sumWeight += set_of_items[i] * weight[i]
    if sumWeight > maxWeight:
        return 0
    else: 
        return sumWeight

def size_checking(set_of_items): #проверка набора на соответствие объёму
    sumSize=0
    for i in range(numGenes):
        sumSize += set_of_items[i] * size[i]
    if sumSize > maxSize:
        return 0
    else: 
        return sumSize

def make_set_of_items_greedy(): #создание набора (жадный выбор со случайного элемента)
    starting_item = rd.randint(0, numGenes-1)
    next_item = starting_item + 1 if starting_item < numGenes - 1 else 0
    set_of_items = [0] * numGenes  
    set_of_items[starting_item] = 1;
    while (size_checking(set_of_items) and weight_checking(set_of_items)):
        set_of_items[next_item] = 1
        next_item = next_item + 1 if next_item < numGenes -1 else 0
    set_of_items[next_item - 1 if next_item != 0 else numGenes - 1] = 0
    return set_of_items


def create_population(): #создание популяции
    population = []
    for x in range(population_size):
        item_set = make_set_of_items_greedy()
        population.append(item_set)
    return population

def set_of_items_value(set_of_items): #определение ценности набора
    sumValue = 0
    for i in range(numGenes):
        sumValue += set_of_items[i] * value[i]
    return sumValue

def select_parents(population): #выбор родителей пропорционально приспособленности
    sum_population_value = 0
    upper_limits = []
    for item_set in population:
        sum_population_value += set_of_items_value(item_set)
        upper_limits.append(sum_population_value)
    parents = []
    first_parent = -1
    while True:
        rnd_value = rd.uniform(0, sum_population_value)
        for i in range(len(upper_limits)):            
            if  rnd_value > upper_limits[i] or upper_limits[i] == sum_population_value:
                if i == first_parent:
                    parents.append(population[i+1 if i < len(upper_limits) - 1 else 0])
                    break
                parents.append(population[i])
                first_parent = i
                break
        if len(parents) == 2:
            break
    return parents
        
#start_population = create_population()
#select_parents(start_population)




In [3]:
def crossingover(first_parent, second_parent): #скрещивание, результат: 2 новых набора
    num_of_kids = 2
    kids = []
    kid = [0]* len(first_parent)
    for k in range(num_of_kids):
        for i in range(len(first_parent)):
            kid[i] = (first_parent[i] if rd.randint(0, 1) == 0 else second_parent[i])
        kids.append(kid)
        kid = [0]* len(first_parent)
    return kids



In [4]:
def mutate_set_of_items(set_of_items,bits): #мутация набора
    if bits >= len(set_of_items):
        return set_of_items
    mutated_bits = []
    i = 0
    while i < bits:
        while True:
            ind = rd.randint(0, len(set_of_items)-1)
            if mutated_bits.count(ind) == 0:
                break
        set_of_items[i] = 1 if set_of_items[i] == 0 else 0
        mutated_bits.append(i)
        i += 1
    return set_of_items

In [5]:
def GA(): #основная процедура
    max_iter = 500 #лимит итераций
    iter_m = max_iter - 1
    start_population = create_population() #стартовая популяция
    start_population.sort(key=lambda x: set_of_items_value(x), reverse=True)
    max_iter -= 1
    
    print("Поколение {gen}".format(gen = 500 - max_iter))
    print("Самый лучший набор {best}".format(best = start_population[0]))
    print("Ценность {val}\n".format(val = set_of_items_value(start_population[0])))
    
    least_value_item_v = 100000 #поиск минимальной ценности из всех грузов
    for vl in value:
        if least_value_item_v > vl:
            least_value_item_v = vl
    
    old_best_score = 0
    current_population = start_population
    for t in range(iter_m): #цикл создания новых популяций
        best_score = set_of_items_value(current_population[0])
        #если ценность набора возросла не более чем на ценность самого дешёвого груза, то остановка
        if old_best_score != best_score and best_score > old_best_score and best_score - old_best_score < least_value_item_v:
            print("Разница между старым лучшим набором и новым лучшим: ",str(best_score - old_best_score) )
            print("Самый 'дешёвый' груз из имеющихся стоит:", str(least_value_item_v), "\n" )
            break;
        #получение списка потомков на основе текущей популяции
        kids = []
        population_parents = []
        for i in range(population_size):
            population_parents.append(current_population[i])
        for i in range(numGenes // 2):
            parents = select_parents(population_parents)
            population_parents.remove(parents[0])
            population_parents.remove(parents[1])
            kid_pair = crossingover(parents[0], parents[1])
            kids.append(kid_pair[0])
            kids.append(kid_pair[1])
        mutated_kids = []
        
        '''for i in range(len(kids)):  
            print(str(kids[i]),'', str(set_of_items_value(kids[i])) )'''
        #мутируем 3 бита у 5% потомства    
        for i in range(len(kids[i]) // 20):
            while True:
                mut_kid = rd.randint(0, len(kids)-1)
                if mutated_kids.count(mut_kid) == 0:
                    kids[mut_kid] = mutate_set_of_items(kids[mut_kid], 3)
                    break
        
        
        '''for i in range(len(kids)):  
            print(str(kids[i]),'', str(set_of_items_value(kids[i])) )'''
        #составляем список стрых наборов с ценностью 90% и их потомков, оставляем лучшие 200 наборов
        curr_pop_with_value = []
        for i in range(len(current_population)):
            curr_pop_with_value.append([current_population[i], set_of_items_value(current_population[i])* 0.9 ] )
        for i in range(len(kids)):
            if weight_checking(kids[i]) and size_checking(kids[i]):
                curr_pop_with_value.append([kids[i], set_of_items_value(kids[i])] )
              
        '''for i in range(200):
            print("цена родителей {gen}".format(gen = curr_pop_with_value[i][1]*0.9))
        for i in range(len(curr_pop_with_value)-200):
            print("цена валидных детей {gen}".format(gen = curr_pop_with_value[i+200][1]))'''
        
        
        
        curr_pop_with_value.sort(key=lambda x: x[1], reverse=True)  
        curr_pop_with_value = curr_pop_with_value[0:population_size]
        
        
        print(curr_pop_with_value[0][1])
        current_population = []
        for i in range(len(curr_pop_with_value)):
            current_population.append(curr_pop_with_value[i][0])
        max_iter -= 1
        curr_pop_with_value = []
        print("Поколение {gen}".format(gen = 500 - max_iter))
        print("Самый лучший набор {best}".format(best = current_population[0]))
        print("Ценность {val}\n".format(val = set_of_items_value(current_population[0])))
        
        old_best_score = best_score
    #выводим содержимое лучшего набора после завершения алгоритма
    sum1,sum2 = 0,0
    table=[]
    for i in range(len(current_population[0])):
        sum1 += current_population[0][i] * size[i]
        sum2 += current_population[0][i] * weight[i]
        if current_population[0][i]==1 :
            th_data={'Индекс':i, 'Вес':weight[i], 'Объём':size[i],  'Ценность':value[i]}
            print(th_data)
    msg="\nГрузоподъемность:"+ str(sum2)+" при лимите:"+str(maxWeight)+"\nВместимость:"+ str(round(sum1))+" при лимите:"+str(maxSize)+"\nЦенность:"+ str(set_of_items_value(current_population[0]))
    print(msg)
        
    
GA()
    

Поколение 1
Самый лучший набор [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Ценность 3926.0

3926.0
Поколение 2
Самый лучший набор [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Ценность 3926.0

3926.0
Поколение 3
Самый лучший набор [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Ценность 3926.0

3986.0
Поколение 4
Самый лучший набор [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0]
Ценность 3986.0

Разница между старым лучшим набором и новым лучшим:  60.0
Самый 'дешёвый' груз из имеющихся стоит: 113.0 

{'Индекс': 10, 'Вес': 458.0, 'Объём': 0.9, 'Ценность': 277.0}
{'Индекс': 11, 'Вес': 1110.0, 'Объём': 0.5, 'Ценность': 351.0}
{'Индекс': 14, 'Вес': 215.0, 'Объём': 0.4, 'Ценность': 373.0}
{'Индекс': 15, 'Вес': 1437.0, 'Объём': 1.2, 'Ценность': 324.0}
{'Индекс': 16, 'Вес': 1690.0, 'Объём': 1.1, 'Ценность': 146.0}
{'И